In [1]:
from skimpy import clean_columns

import os
import pandas as pd
import numpy as np
import time

import rasterio
import planetary_computer as pc
import geopandas as gpd

from scipy.ndimage import maximum_filter as maxf2D
from scipy.ndimage import minimum_filter as minf2D
from scipy.ndimage import convolve as conf2D

from shapely.geometry import Point
from rasterio.crs import CRS

import data_sampling_workflow.sample_rasters as sr
import raster_to_features as rf

In [2]:
year = 2020

In [3]:
# convert reference class columns in df from CollectEarth 
# into a single ref_class column with values from 0 to 3
def ref_class_column(df):

    map_class = df.pl_class
    ref_class = []

    for i in map_class.index:
        if df.category_non_iceplant_vegetation.loc[i] == 100:
            ref_class.append(0)
        elif df.category_iceplant.loc[i] == 100:
            ref_class.append(1)
        elif df.category_low_ndvi_impervious_surface.loc[i] == 100:
            ref_class.append(2)
        elif df.category_water.loc[i] == 100:
            ref_class.append(3)
        else:
            ref_class[j]= 100
            
    return ref_class

In [7]:
# ---------------------------------------------------
# load validation points
fp = os.path.join(os.getcwd(), 'map_validation', 'validation_results_'+str(year)+'.csv' )
validation_pts = clean_columns(sr.geodataframe_from_csv(fp = fp, 
                                                        lon_label = 'center_lon', 
                                                        lat_label = 'center_lat', 
                                                        crs = CRS.from_epsg(4326)))

# clean data frame 
validation_pts['ref_class'] = ref_class_column(validation_pts)
validation_pts = validation_pts.drop(['center_lon', 'center_lat', 'shape', 'size_m', 'sample_points',
        'flagged', 'flagged_reason',  
         'total_securewatch_dates', 'common_securewatch_date', 
         'validation_finished_yes_high_confidence',
         'collection_time',
         'category_low_ndvi_impervious_surface','category_non_iceplant_vegetation',
         'category_iceplant', 'category_water', 'plotid'], axis =1)
validation_pts = validation_pts.rename( 
    columns = {'pl_class':'map_class',
               'validation_finished_no_low_confidence':'low_confidence'})

# ---------------------------------------------------
# load NAIP scenes' footprints
footprints = gpd.read_file(os.path.join(os.getcwd(), 
                                        'separating_naip_flights',
                                        'naip_scenes_footprints',
                                        'naip_scenes_footprints.shp'))
footprints = footprints[footprints.year == str(year)]
footprints.reset_index(inplace=True, drop = True)

In [26]:
# ---------------------------------------------------
# make dictionary with points that need to be sampled from each scene
unchecked = list(validation_pts.index)
pts_to_sample = {key: [] for key in footprints.id}

for itemid in footprints.id:
    box = list(footprints[footprints.id == itemid].geometry)[0]
    
    to_remove = []
    for i in unchecked:
        point = validation_pts.iloc[i].geometry
        if box.contains(point) == True:
            pts_to_sample[itemid].append(i)
            to_remove.append(i)
            
    for i in to_remove:
        unchecked.remove(i)

In [11]:
# ---------------------------------------------------
# Open canopy height raster and create auxiliary min, max, and avg rasters
lidar_rast_r = rasterio.open(sr.path_to_lidar(year))
rast_name = 'SB_canopy_height_' + str(year) # give a name to read raster
fp = os.path.join(os.getcwd(), 'temp', 'aux_canopy_height')

sr.max_min_avg_rasters(rast_reader = lidar_rast_r, rast_name = rast_name, n=3, folder_path=fp)

# file paths to auxiliary canopy height rasters
lidar_fps = []
for tag in ['_maxs', '_mins', '_avgs']:
    lidar_fps.append(os.path.join(fp, rast_name + tag + '.tif'))

In [28]:
sampled_points = []
#for itemid in samples:
for itemid in ['ca_m_3412037_nw_10_060_20200607']:
    if len(pts_to_sample[itemid]) > 0:

        # ---------------------------------------------------        
        # sample spectral bands from NAIP
        item = sr.get_item_from_id(itemid)
        scene_rast_r = sr.get_raster_from_item(item)     

        to_sample = validation_pts.iloc[pts_to_sample[itemid]].geometry
        to_sample_match = to_sample.to_crs(scene_rast_r.crs)

        rast_band_names = ['r', 'g', 'b', 'nir']
        spectral_bands = sr.sample_raster_from_pts(to_sample_match, scene_rast_r, rast_band_names).set_index(to_sample.index)

        # ---------------------------------------------------
        # sample canopy_height at point, and max, min and avg canopy height around point        
        to_sample_match = to_sample.to_crs(lidar_rast_r.crs)

        canopy_h_samples = [sr.sample_raster_from_pts(to_sample_match, lidar_rast_r, ['lidar'])]
        for i, tag in zip(range(3),['min', 'max', 'avg']):
            aux_rast_r = rasterio.open(lidar_fps[i])
            canopy_h_samples.append(sr.sample_raster_from_pts(to_sample_match, aux_rast_r, [tag+'_lidar']))

        # ---------------------------------------------------
        # concatenate sampled data
        lidar_bands = pd.concat(canopy_h_samples, axis=1).set_index(to_sample.index)
        df = pd.concat([to_sample, spectral_bands, lidar_bands], axis=1)
      
        # ---------------------------------------------------
        # add date and naipid information
        kwargs = {'year' : item.datetime.year,
                  'month' : item.datetime.month,
                  'day_in_year' : sr.day_in_year(item.datetime.day, item.datetime.month, item.datetime.year),
                  'naip_id' : itemid}
        df = df.assign(**kwargs)
                
        # ---------------------------------------------------
        sampled_points.append(gpd.GeoDataFrame(df))

In [23]:
samples = pd.concat(sampled_points).sort_index()
samples = pd.concat([samples, validation_pts.drop(['geometry'], axis=1)], axis=1)

samples = samples[['geometry', 
 'r', 'g', 'b', 'nir', 
 'lidar', 'max_lidar', 'min_lidar', 'avg_lidar', 
 'year', 'month', 'day_in_year', 
 'naip_id', 'pl_which_raster',
 'email', 'analysis_duration','low_confidence',
  'map_class', 'ref_class']]

samples

,geometry,r,g,b,nir,lidar,max_lidar,min_lidar,avg_lidar,year,month,day_in_year,naip_id,pl_which_raster,email,analysis_duration,low_confidence,map_class,ref_class
8,POINT (-120.47411 34.45362),211.0,211.0,210.0,159.0,-9999.0,-9999.0,-9999.0,-2717.222168,2020.0,6.0,159.0,ca_m_3412037_nw_10_060_20200607,0,galaz-garcia@nceas.ucsb.edu,5.8 secs,0.0,2,3
12,POINT (-120.46838 34.45008),135.0,98.0,78.0,161.0,0.0,0.0,0.0,0.000000,2020.0,6.0,159.0,ca_m_3412037_nw_10_060_20200607,0,galaz-garcia@nceas.ucsb.edu,7.0 secs,0.0,1,1
18,POINT (-120.45116 34.44341),61.0,68.0,80.0,14.0,-9999.0,-9999.0,-9999.0,-2717.222168,2020.0,6.0,159.0,ca_m_3412037_nw_10_060_20200607,0,galaz-garcia@nceas.ucsb.edu,5.9 secs,0.0,3,3
30,POINT (-120.46747 34.45950),110.0,84.0,75.0,137.0,0.0,0.0,0.0,0.000000,2020.0,6.0,159.0,ca_m_3412037_nw_10_060_20200607,0,brun@nceas.ucsb.edu,9.5 secs,0.0,1,1
35,POINT (-120.47336 34.45746),139.0,125.0,101.0,73.0,-9999.0,-9999.0,0.0,615.777771,2020.0,6.0,159.0,ca_m_3412037_nw_10_060_20200607,0,brun@nceas.ucsb.edu,13.2 secs,0.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,galaz-garcia@nceas.ucsb.edu,6.3 secs,0.0,0,0
461,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,galaz-garcia@nceas.ucsb.edu,4.7 secs,0.0,2,2
462,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,galaz-garcia@nceas.ucsb.edu,10.8 secs,0.0,0,0
463,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,galaz-garcia@nceas.ucsb.edu,10.3 secs,0.0,2,0


In [6]:
# samples = pd.concat(sampled_points).sort_index()
# samples = pd.concat([samples, validation_pts.drop(['geometry'], axis=1)], axis=1)

# samples = samples[['geometry', 
#  'r', 'g', 'b', 'nir', 
#  'lidar', 'max_lidar', 'min_lidar', 'avg_lidar', 
#  'year', 'month', 'day_in_year', 
#  'naip_id', 'pl_which_raster',
#  'email', 'analysis_duration','low_confidence',
#   'map_class', 'ref_class']]

# samples

,geometry,r,g,b,nir,lidar,max_lidar,min_lidar,avg_lidar,year,month,day_in_year,naip_id,pl_which_raster,email,analysis_duration,low_confidence,map_class,ref_class
0,POINT (-120.37323 34.45406),114,135,144,36,-9999,-9999,-9999,-2717.222168,2020,6,159,ca_m_3412037_ne_10_060_20200607,0,galaz-garcia@nceas.ucsb.edu,9.3 secs,0.0,3,3
1,POINT (-119.58537 34.42428),91,98,81,154,2,4,0,2.111111,2020,5,142,ca_m_3411936_sw_11_060_20200521,2,galaz-garcia@nceas.ucsb.edu,5.9 secs,0.0,1,0
2,POINT (-120.16952 34.47180),109,114,104,131,1,3,0,1.111111,2020,5,143,ca_m_3412039_ne_10_060_20200522,0,galaz-garcia@nceas.ucsb.edu,222.9 secs,0.0,0,0
3,POINT (-120.00898 34.46674),103,119,109,159,2,3,1,1.777778,2020,5,143,ca_m_3412040_ne_10_060_20200522,0,galaz-garcia@nceas.ucsb.edu,155.6 secs,0.0,1,0
4,POINT (-119.97703 34.45705),123,130,107,155,0,2,0,0.555556,2020,5,143,ca_m_3411933_nw_11_060_20200522,2,galaz-garcia@nceas.ucsb.edu,155.7 secs,100.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,POINT (-120.34612 34.46073),23,33,48,31,4,5,1,3.555556,2020,5,144,ca_m_3412038_nw_10_060_20200523,0,galaz-garcia@nceas.ucsb.edu,6.3 secs,0.0,0,0
461,POINT (-120.14253 34.47797),191,179,159,182,0,0,0,0.000000,2020,5,143,ca_m_3412039_ne_10_060_20200522,0,galaz-garcia@nceas.ucsb.edu,4.7 secs,0.0,2,2
462,POINT (-119.85058 34.40829),68,83,72,145,8,13,5,8.555555,2020,5,142,ca_m_3411934_sw_11_060_20200521,2,galaz-garcia@nceas.ucsb.edu,10.8 secs,0.0,0,0
463,POINT (-120.27393 34.47502),61,67,71,64,1,2,1,1.111111,2020,5,143,ca_m_3412038_ne_10_060_20200522,0,galaz-garcia@nceas.ucsb.edu,10.3 secs,0.0,2,0


In [7]:
folder = os.path.join(os.getcwd(), 
                      'validation_augmented_spectral_'+str(year))
if os.path.exists(folder) == False:
    os.mkdir(folder)

samples.to_file(os.path.join(folder, 'validation_augmented_spectral_'+str(year))+'.shp')

/tmp/ipykernel_2347/1547652901.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  samples.to_file(os.path.join(folder, 'validation_results_spectral_'+str(year))+'.shp')


In [2]:
# *********************************************************************

def min_raster(rast_reader, band, rast_name, n, folder_path=''):  
    """
        Creates a new raster by replacing each pixel p in given raster R by the minimum value in a nxn window centered at p.
        The raster with minimum values is saved in a temp folder in the current working directory if no folder_path is given.
            Parameters: 
                        rast_reader (rasterio.io.DatasetReader):
                            reader to the raster from which to compute the minimum values in a window
                        rast_name (str):
                            name of raster. The resulting raster will be saved as rast_name_maxs.tif.
                        n (int):
                            Side length (in pixels) of the square window over which to compute minimum values for each pixel.
                        folder_path (str):
                            directory where to save raster. If none is given, then it saves the raster in a temp folder in the cwd.
            Return: None    
    """
    rast = rast_reader.read([band]).squeeze() # read raster values
    mins = minf2D(rast, size=(n,n))    # calculate min in window
    
    if not folder_path:                         # if needed, create temp directory to save files 
        folder_path = make_directory('temp')
    
    dtype = rasterio.dtypes.get_minimum_dtype(mins)  # parameters for saving
    
    fp = os.path.join(folder_path, rast_name +'_mins.tif')      # save raster
    sr.save_raster(mins, 
                fp, 
                rast.shape,
                1,
                rast_reader.crs, 
                rast_reader.transform, 
                dtype)  
    return

# ------------------------------------------------------------------------------

def max_raster(rast_reader, band, rast_name, n, folder_path=''):  
    """
        Creates a new raster by replacing each pixel p in given raster R by the max value in a nxn window centered at p.
        The raster with maximum values is saved in a temp folder in the current working directory if no folder_path is given.
            Parameters: 
                        rast_reader (rasterio.io.DatasetReader):
                            reader to the raster from which to compute the maximum values in a window
                        rast_name (str):
                            name of raster. The resulting raster will be saved as rast_name_maxs.tif.
                        n (int):
                            Side length (in pixels) of the square window over which to compute maximum values for each pixel.
                        folder_path (str):
                            directory where to save raster. If none is given, then it saves the raster in a temp folder in the cwd.
            Return: None    
    """
    rast = rast_reader.read([band]).squeeze() # read raster values
    maxs = maxf2D(rast, size=(n,n))    # calculate min in window
    
    if not folder_path:                         # if needed, create temp directory to save files 
        folder_path = make_directory('temp')
    
    dtype = rasterio.dtypes.get_minimum_dtype(maxs)  # parameters for saving
    
    fp = os.path.join(folder_path, rast_name +'_maxs.tif')      # save raster
    sr.save_raster(maxs, 
                fp, 
                rast.shape,
                1,
                rast_reader.crs, 
                rast_reader.transform, 
                dtype)  
    return

# ------------------------------------------------------------------------------

def avg_raster(rast_reader, band, rast_name, n, folder_path=''): 
    """
        Creates a new raster by replacing each pixel p in given raster R by the avg value in a nxn window centered at p.
        The raster with averege values is saved in a temp folder in the current working directory if no folder_path is given.
            Parameters: 
                        rast_reader (rasterio.io.DatasetReader):
                            reader to the raster from which to compute the average values in a window
                        rast_name (str):
                            name of raster. The resulting raster will be saved as rast_name_avgs.tif.
                        n (int):
                            Side length (in pixels) of the square window over which to compute average values for each pixel.
                        folder_path (str):
                            directory where to save raster. If none is given, then it saves the raster in a temp folder in the cwd.
            Return: None    
    """
    rast = rast_reader.read([band]).squeeze() # read raster values

    w = np.ones(n*n).reshape(n,n)      # calculate averages in window
    avgs = conf2D(rast, 
             weights=w,
             mode='constant')
    avgs = avgs/(n*n)
    
    # if needed, create temp directory to save files 
    if not folder_path:  
        folder_path = make_directory('temp')
            
    # parameters for saving   
    fp = os.path.join(folder_path, rast_name +'_avgs.tif')                
    dtype = rasterio.dtypes.get_minimum_dtype(avgs)
            
    sr.save_raster(avgs,    # save rasters
                fp, 
                rast.shape, 
                1,
                rast_reader.crs, 
                rast_reader.transform, 
                dtype)  
    return

In [3]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

#itemids = pd.read_csv(os.path.join(os.getcwd(),'temp', 'aoi_naip_itemids.csv')).itemid

#csv_name = 'test_set.csv'
df = gpd.read_file(os.path.join(os.getcwd(), 'validation_results_spectral_2020', 'validation_results_spectral_2020.shp'))
itemids = list(df.naip_id.unique())
# ***************************************************

In [4]:
pts_list = []

for itemid in itemids:
    print('PROCESSING: ', itemid)
    # ***************************************************
    # ------------------------------
    # Open NAIP scene and calculate auxiliary spectral rasters
    item = sr.get_item_from_id(itemid)    # locate raster
    naip_rast_r = sr.get_raster_from_item(item)

    folp = os.path.join(os.getcwd(),'temp','aux_naip_rasters')
    if os.path.exists(folp) == False:
        os.mkdir(folp)

    t0 = time.time()
    rast_name = 'r_'+itemid 
    min_raster(rast_reader = naip_rast_r, band=1, rast_name = rast_name, n=3, folder_path=folp)
    max_raster(rast_reader = naip_rast_r, band=1, rast_name = rast_name, n=3, folder_path=folp)
    avg_raster(rast_reader = naip_rast_r, band=1, rast_name = rast_name, n=3, folder_path=folp)

    rast_name = 'g_'+itemid 
    min_raster(rast_reader = naip_rast_r, band=2, rast_name = rast_name, n=3, folder_path=folp)
    max_raster(rast_reader = naip_rast_r, band=2, rast_name = rast_name, n=3, folder_path=folp)
    avg_raster(rast_reader = naip_rast_r, band=2, rast_name = rast_name, n=3, folder_path=folp)

    rast_name = 'b_'+itemid 
    min_raster(rast_reader = naip_rast_r, band=3, rast_name = rast_name, n=3, folder_path=folp)
    max_raster(rast_reader = naip_rast_r, band=3, rast_name = rast_name, n=3, folder_path=folp)
    avg_raster(rast_reader = naip_rast_r, band=3, rast_name = rast_name, n=3, folder_path=folp)

    rast_name = 'nir_'+itemid 
    min_raster(rast_reader = naip_rast_r, band=4, rast_name = rast_name, n=3, folder_path=folp)
    max_raster(rast_reader = naip_rast_r, band=4, rast_name = rast_name, n=3, folder_path=folp)
    avg_raster(rast_reader = naip_rast_r, band=4, rast_name = rast_name, n=3, folder_path=folp)
    
    print('CREATED RASTERS (sec): ',time.time() -t0)

    # ***************************************************
    # Find points in test and train sets with this itemid
    pts = df.loc[df['naip_id'] == itemid]
    pts_col = pts.to_crs(naip_rast_r.crs).geometry


    # ***************************************************
    ## Sample max, min and avg spectral info around point
    t0 = time.time()
    samples = []
    for band in ['r_','g_','b_','nir_']:
        for tag in ['_maxs', '_mins', '_avgs']:

            fp = os.path.join(folp,  band + itemid + tag + '.tif')
            col_name = band.replace('_', '') + tag.replace('s','')

            rast_r = rasterio.open(fp)
            sample = sr.sample_raster_from_pts(pts_col, rast_r, [col_name])
    
            samples.append(sample)

    print('SAMPLED RASTERS (sec): ',time.time() -t0)
    new_features = pd.concat(samples, axis = 1)
    
    # ------------------------------
    ## Add all derived spectral data to pts dataframe

    pts = pd.concat([pts, new_features.set_index(pts.index)], axis=1)    
    
    # ***************************************************
    # create difference in window columns
    for band in ['r_','g_','b_','nir_']:
        col_name = band + 'diff'
        pts[col_name] = pts[band +'max'] - pts[band +'min']

    pts['diff_lidar'] = pts['max_lidar'] - pts['min_lidar']
    
    pts['ndvi'] = rf.normalized_difference_index(df, 3,0)


    # ***************************************************
    # Clean dataframe
    pts = pts[['geometry', #  point location
              'naip_id',   # sampling info
             'r', 'r_max', 'r_min', 'r_diff', 'r_avg',
             'g', 'g_max', 'g_min', 'g_diff', 'g_avg',
             'b', 'b_max', 'b_min', 'b_diff', 'b_avg',
             'nir', 'nir_max', 'nir_min', 'nir_diff', 'nir_avg',
             'ndvi',     # spectral
             'year', 'month', 'day_in_yea', # date
             'lidar', 'max_lidar', 'min_lidar', 'diff_lidar', 'avg_lidar', # lidar
             'map_class', 'ref_class'
             ]] 

    # ***************************************************
    # Delete auxiliary NAIP rasters created for this scene
    for band in ['r_','g_','b_','nir_']:
        for tag in ['_maxs', '_mins', '_avgs']:
            fp = os.path.join(folp, band + itemid + tag + '.tif')   
            os.remove(fp)

    pts_list.append(pts)
    
# ***************************************************    
    
all_pts = pd.concat(pts_list, axis =0)
os.rmdir(folp)

PROCESSING:  ca_m_3412037_ne_10_060_20200607
CREATED RASTERS (sec):  65.20446634292603
SAMPLED RASTERS (sec):  0.06897330284118652
PROCESSING:  ca_m_3411936_sw_11_060_20200521
CREATED RASTERS (sec):  64.61776399612427
SAMPLED RASTERS (sec):  0.12017083168029785
PROCESSING:  ca_m_3412039_ne_10_060_20200522
CREATED RASTERS (sec):  69.72652339935303
SAMPLED RASTERS (sec):  0.12414717674255371
PROCESSING:  ca_m_3412040_ne_10_060_20200522
CREATED RASTERS (sec):  66.93128371238708
SAMPLED RASTERS (sec):  0.12166118621826172
PROCESSING:  ca_m_3411933_nw_11_060_20200522
CREATED RASTERS (sec):  68.73958992958069
SAMPLED RASTERS (sec):  0.12128043174743652
PROCESSING:  ca_m_3411935_sw_11_060_20200521
CREATED RASTERS (sec):  65.98848676681519
SAMPLED RASTERS (sec):  0.1220860481262207
PROCESSING:  ca_m_3412038_ne_10_060_20200522
CREATED RASTERS (sec):  68.30569815635681
SAMPLED RASTERS (sec):  0.11563920974731445
PROCESSING:  ca_m_3411933_se_11_060_20200521
CREATED RASTERS (sec):  58.241414785385

In [5]:
train = pd.read_csv(os.path.join(os.getcwd(),'data_sampling_workflow','spectral_window_train_set.csv'))
train.head()

,x,y,pts_crs,aoi,naip_id,polygon_id,r,r_max,r_min,r_diff,...,ndvi,year,month,day_in_year,lidar,max_lidar,min_lidar,min_max_diff,avg_lidar,iceplant
0,732565.038693,3.815895e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,15,107,113,105,8,...,0.235714,2020,6,159,0.0,4.0,0.0,4.0,0.777778,1.0
1,733735.631738,3.814310e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,6,123,127,113,14,...,0.127660,2020,6,159,0.0,0.0,0.0,0.0,0.000000,1.0
2,732426.543981,3.815916e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,14,109,109,103,6,...,0.221429,2020,6,159,1.0,6.0,0.0,6.0,1.555556,1.0
3,732609.439595,3.815869e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,15,92,94,87,7,...,0.321033,2020,6,159,3.0,3.0,0.0,3.0,2.111111,1.0
4,733129.129485,3.814913e+06,epsg:26910,point_conception,ca_m_3412037_nw_10_060_20200607,52,83,83,79,4,...,0.333333,2020,6,159,0.0,3.0,0.0,3.0,1.888889,1.0


In [6]:
train.columns

Index(['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'polygon_id', 'r', 'r_max',
       'r_min', 'r_diff', 'r_avg', 'g', 'g_max', 'g_min', 'g_diff', 'g_avg',
       'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'nir', 'nir_max', 'nir_min',
       'nir_diff', 'nir_avg', 'ndvi', 'year', 'month', 'day_in_year', 'lidar',
       'max_lidar', 'min_lidar', 'min_max_diff', 'avg_lidar', 'iceplant'],
      dtype='object')

In [8]:
all_pts.sort_index()

,geometry,naip_id,r,r_max,r_min,r_diff,r_avg,g,g_max,g_min,...,year,month,day_in_yea,lidar,max_lidar,min_lidar,diff_lidar,avg_lidar,map_class,ref_class
0,POINT (-120.37323 34.45406),ca_m_3412037_ne_10_060_20200607,114,145,83,62,26.111111,135,160,112,...,2020,6,159,-9999,-9999,-9999,0,-2717.222168,3,3
1,POINT (-119.58537 34.42428),ca_m_3411936_sw_11_060_20200521,91,97,77,20,2.000000,98,103,92,...,2020,5,142,2,4,0,4,2.111111,1,0
2,POINT (-120.16952 34.47180),ca_m_3412039_ne_10_060_20200522,109,112,107,5,24.111111,114,122,114,...,2020,5,143,1,3,0,3,1.111111,0,0
3,POINT (-120.00898 34.46674),ca_m_3412040_ne_10_060_20200522,103,116,101,15,23.555555,119,137,115,...,2020,5,143,2,3,1,2,1.777778,1,0
4,POINT (-119.97703 34.45705),ca_m_3411933_nw_11_060_20200522,123,142,116,26,15.666667,130,142,122,...,2020,5,143,0,2,0,2,0.555556,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,POINT (-120.34612 34.46073),ca_m_3412038_nw_10_060_20200523,23,51,19,32,0.777778,33,68,29,...,2020,5,144,4,5,1,4,3.555556,0,0
461,POINT (-120.14253 34.47797),ca_m_3412039_ne_10_060_20200522,191,191,184,7,18.000000,179,179,167,...,2020,5,143,0,0,0,0,0.000000,2,2
462,POINT (-119.85058 34.40829),ca_m_3411934_sw_11_060_20200521,68,72,52,20,7.222222,83,88,75,...,2020,5,142,8,13,5,8,8.555555,0,0
463,POINT (-120.27393 34.47502),ca_m_3412038_ne_10_060_20200522,61,82,61,21,9.666667,67,89,67,...,2020,5,143,1,2,1,1,1.111111,2,0


In [10]:
all_pts.to_csv(os.path.join(os.getcwd(), 'validation_results_spectral_2020', 'validation_results_spectral_2020.csv'), ignore_index = True)